In [33]:
#讀movie資料
import pandas as pd
df_train = pd.read_csv("train_v2.csv")
df_test = pd.read_csv("test_v2.csv")

In [34]:
df_train=df_train[df_train.index<10000]

In [35]:
df_train.head()

,ID,user,movie,rating
0,610739,3704,3784,3
1,324753,1924,802,3
2,808218,4837,1387,4
3,133808,867,1196,4
4,431858,2631,3072,5


In [36]:
train_features=df_train[['ID','user','movie']]
target=df_train['rating']

In [37]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_features,target,test_size=0.3,random_state=123)

## XLearn

In [ ]:
#三種FM系列資料格式（xlearn皆可吃）
#<libsvm format>:
#   label index_1:value_1 index_2:value_2 ... index_n:value_n
#<CSV format>:
#   value_1 value_2 .. value_n label
#<libffm format>:
#   label field_1:index_1:value_1 field_2:index_2:value_2 ...

In [6]:
#看起來他的lr模型只支援0/1分類問題  但可以直接吃dataframe格式
import xlearn as xl
linear_model = xl.LRModel(task='binary', init=0.1,
                          epoch=10, lr=0.1,
                          reg_lambda=1.0, opt='sgd')
linear_model.fit(x_train, y_train,
                 eval_set=[x_test, y_test],
                 is_lock_free=False)
# Generate predictions
y_pred = linear_model.predict(df_test)

/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/feature_extraction/hashing.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _hashing
/anaconda2/envs/tensorflow/lib/python2.7/site-packages/scipy/io/matlab/mio4.py:18: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio_utils import squeeze_element, chars_to_strings
/anaconda2/envs/tensorflow/lib/python2.7/site-packages/scipy/io/matlab/mio5.py:98: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio5_utils import VarReader5
/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/datasets/svmlight_format.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._svmlight_format import _load_svmlight_file


In [6]:
# 需把dataframe的資料轉成FFM架構他才能吃
def convert_to_ffm(df,type,numerics,categories,features):
    currentcode = len(numerics)
    catdict = {}
    catcodes = {}
    # Flagging categorical and numerical fields
    for x in numerics:
         catdict[x] = 0
    for x in categories:
         catdict[x] = 1
    
    nrows = df.shape[0]
    ncolumns = len(features)
    with open(str(type) + "_ffm.txt", "w") as text_file:
# Looping over rows to convert each row to libffm format
        for n,r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow['rating']))
             # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(catdict.keys()):
                if(catdict[x]==0):
                    datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
                else:
            # For a new field appearing in a training example
                    if(x not in catcodes):
                        catcodes[x] = {}
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
            # For already encoded fields
                    elif(datarow[x] not in catcodes[x]):
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
                    code = catcodes[x][datarow[x]]
                    datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

In [7]:
train=pd.concat([x_train,y_train],axis=1)
vali=pd.concat([x_test,y_test],axis=1)
#找出各型態資料轉換成FFM資料格式
num_col1 = pd.DataFrame([])
cat_col1 = train.iloc[:,1:3]
all_col1 = num_col1 + cat_col1
convert_to_ffm(train,'Train',num_col1,cat_col1,all_col1)

num_col2 = pd.DataFrame([])
cat_col2 = vali.iloc[:,1:3]
all_col2 = num_col2 + cat_col2
convert_to_ffm(vali,'Vali',num_col2,cat_col2,all_col2)

In [8]:
import xlearn as xl
ffm_model = xl.create_fm()  # Use field-aware factorization machine
ffm_model.setTrain("Train_ffm.txt")   # Training data
ffm_model.setValidate("Vali_ffm.txt")  # Validation data
param = {'task':'reg', 'lr':0.2, 
         'lambda':0.002, 'metric':'rmse'}
#param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
#         'k':10,           # Size of latent factor 
#         'lr':0.1,        # Learning rate for GD
#         'lambda':0.0002, # L2 Regularization Parameter
#         'metric':'auc',  # Metric for monitoring validation set performance
#         'epoch':25       # Maximum number of Epochs
#        }
ffm_model.fit(param,'model.out')

/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/feature_extraction/hashing.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _hashing
/anaconda2/envs/tensorflow/lib/python2.7/site-packages/scipy/io/matlab/mio4.py:18: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio_utils import squeeze_element, chars_to_strings
/anaconda2/envs/tensorflow/lib/python2.7/site-packages/scipy/io/matlab/mio5.py:98: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio5_utils import VarReader5
/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/datasets/svmlight_format.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._svmlight_format import _load_svmlight_file


In [9]:
#test資料格式也要轉
#先加一欄都為0的rating
import numpy as np
r0=pd.DataFrame(np.zeros(250053),columns=['rating'])
test=pd.concat([df_test,r0],axis=1)
test=test[test.index<1000]

In [10]:
#test資料格式也要轉
num_col3 = pd.DataFrame([])
cat_col3 = test.iloc[:,1:3]
all_col3 = num_col3 + cat_col3
convert_to_ffm(test,'Test',num_col3,cat_col3,all_col3)

In [74]:
# Prediction task
#ffm_model.setTest("Test_ffm.txt")  # Test data
ffm_model.setTest("Vali_ffm.txt")  # Vali data
#ffm_model.setSigmoid()  # Convert output to 0-1

# Start to predict
# The output result will be stored in output.txt
ffm_model.predict("model.out", "output.txt")

In [96]:
s=pd.read_csv("output.txt",header=None)
s.columns=['S1']
ss1=np.array(s.S1.tolist())
ss2=np.array(y_test.tolist())
#RMSE
np.mean((ss2 - ss1) ** 2) ** 0.5

1.1313323384489193

## tffm

In [13]:
np.__path__

['/anaconda2/envs/tensorflow/lib/python2.7/site-packages/numpy']

In [41]:
ratings = df_train['rating'].values
df_train2 = df_train.drop('rating', axis=1).values

In [42]:
X_tr, X_te, y_tr, y_te = train_test_split(df_train, ratings, random_state=42, test_size=0.3)

In [43]:
# Convert data to vector format used in FM using one-hot encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore').fit(X_tr)
X_tr = encoder.transform(X_tr)
X_te = encoder.transform(X_te)

In [39]:
import time
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, accuracy_score

In [97]:
import tensorflow as tf
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error
order=3
model = TFFMRegressor(
    order=order, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=60, 
    batch_size=500,
    init_std=0.001,
    reg=0.01,
    input_type='sparse'
)
# translate Y from {0,1} to {-10, 10}
model.fit(X_tr, y_tr, show_progress=True)
predictions = model.predict(X_te)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_te, predictions > 0)))
print('MSE: {}'.format(mean_squared_error(y_te, predictions)))
print('RMSE: {}'.format(np.mean((y_te - predictions) ** 2) ** 0.5))
model.destroy()

100%|██████████| 60/60 [00:09<00:00,  6.57epoch/s]


[order=3] accuracy: 0.0566666666667
MSE: 3.17464695373
RMSE: 1.78175389819


## pywFM

In [57]:
import pywFM

In [58]:
LIBFM_PATH = '/Users/gansulab/Desktop/Rec_Learning/Gansulab/libfm/bin/'

import sys
import os

sys.path.insert(0, LIBFM_PATH)
os.environ["PATH"] += os.pathsep + LIBFM_PATH
os.environ["LIBFM_PATH"] = LIBFM_PATH
    
print os.environ.get('LIBFM_PATH')

/Users/gansulab/Desktop/Rec_Learning/Gansulab/libfm/bin/


In [59]:
X_tr.toarray()

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [61]:
fm_dict_based_on_stdev = {}
# std_dev = 0.1
fm = pywFM.FM(task='regression', num_iter=5, init_stdev=0.1)
fm_dict_based_on_stdev['0.1'] = fm

# std_dev = 0.2
fm = pywFM.FM(task='regression', num_iter=5, init_stdev=0.2)
fm_dict_based_on_stdev['0.2'] = fm

# std_dev = 0.5
fm = pywFM.FM(task='regression', num_iter=5, init_stdev=0.5)
fm_dict_based_on_stdev['0.5'] = fm

# std_dev = 1.0
fm = pywFM.FM(task='regression', num_iter=5, init_stdev=1.0)
fm_dict_based_on_stdev['1.0'] = fm

In [67]:
results = pd.DataFrame()
for ele in fm_dict_based_on_stdev:
    fm = fm_dict_based_on_stdev[ele]
    model = fm.run(X_tr, y_tr, X_te, y_te)
    predictions = model.predictions
    results.ix['std_dev=' + ele, 'RMSE'] = np.mean((y_te - predictions) ** 2) ** 0.5
results

1551


/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


1526
1543
1543


,RMSE
std_dev=0.1,0.455827
std_dev=1.0,0.788337
std_dev=0.2,0.322246
std_dev=0.5,0.419594
